# Create OCEAN Data NFT
Quickly create OCEAN Data NFTs by querying blockchain data from Dune and Flipside.

**Steps:**
1. Get Data
2. Store Data
3. Create Data NFT

In [1]:
import json
import pandas as pd
import os
import time
from dotenv import load_dotenv
import requests
import subprocess
from flipside import Flipside
from ocean_lib.web3_internal.utils import connect_to_network
from ocean_lib.example_config import get_config_dict
from ocean_lib.ocean.ocean import Ocean
from brownie.network import accounts, chain

# Load variables from .env file
load_dotenv()
DUNE_API_KEY = os.getenv("DUNE_API_KEY")
FLIPSIDE_API_KEY = os.getenv("FLIPSIDE_API_KEY")
WEATHER_API_KEY = os.getenv("WEATHER_API_KEY")

## 1. Get Data
### Flipside

* SDK Docs: https://docs.flipsidecrypto.com/flipside-api/get-started

In [2]:
# Query Flipside using their Python SDK
def query_flipside(sql):
    flipside = Flipside(FLIPSIDE_API_KEY, "https://api-v2.flipsidecrypto.xyz")
    results = flipside.query(sql)
    results_df = pd.DataFrame(results.rows, columns=results.columns).drop(
        columns=["__row_index"]
    )
    return results_df

In [3]:
sql = """
SELECT
  date(block_timestamp) as dt,
  count(distinct tx_hash) as tx_ct
FROM ethereum.core.fact_transactions
WHERE block_timestamp >= GETDATE() - interval'7 days'
GROUP BY 1
order by 1 asc
"""
results_df = query_flipside(sql)
results_df

,dt,tx_ct
0,2023-08-28T00:00:00.000Z,306382
1,2023-08-29T00:00:00.000Z,1019692
2,2023-08-30T00:00:00.000Z,1027316
3,2023-08-31T00:00:00.000Z,1010128
4,2023-09-01T00:00:00.000Z,1006635
5,2023-09-02T00:00:00.000Z,1024172
6,2023-09-03T00:00:00.000Z,1038128
7,2023-09-04T00:00:00.000Z,661811


### Dune

* API Docs: https://dune.com/docs/api/

In [6]:
# Query Dune Analytics via API
def query_dune(q):
    url = f"https://api.dune.com/api/v1/query/{q}/results?api_key={DUNE_API_KEY}"
    response = requests.get(url)
    results_json = json.loads(response.text)["result"]["rows"]
    results_df = pd.DataFrame.from_dict(results_json)
    return results_df

In [7]:
dune_query_id = 2847682  # Query URL: https://dune.com/queries/2847682
results_df = query_dune(dune_query_id)
results_df

KeyError: 'result'

### Weather API

* API Docs: https://www.visualcrossing.com/resources/documentation/weather-api/timeline-weather-api/

In [27]:
weather_url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/London%2CUK/2022-07-15/2022-07-20?unitGroup=metric&contentType=csv&include=days&key={WEATHER_API_KEY}"

results_df = pd.read_csv(weather_url)

results_df

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,"London, England, United Kingdom",2022-07-15,25.5,14.6,20.6,25.5,14.6,20.6,8.3,46.5,...,17.8,9,NaN,2022-07-15T05:00:53,2022-07-15T21:11:28,0.56,Clear,Clear conditions throughout the day.,clear-day,"03769099999,03680099999,D5621,03781099999,0367..."
1,"London, England, United Kingdom",2022-07-16,27.8,15.3,21.8,27.1,15.3,21.7,10.9,51.8,...,17.5,8,NaN,2022-07-16T05:02:05,2022-07-16T21:10:27,0.59,Clear,Clear conditions throughout the day.,clear-day,"03769099999,03680099999,D5621,03781099999,0367..."
2,"London, England, United Kingdom",2022-07-17,30.0,16.1,23.6,28.7,16.1,23.2,9.1,42.2,...,17.0,8,NaN,2022-07-17T05:03:18,2022-07-17T21:09:23,0.62,Clear,Clear conditions throughout the day.,clear-day,"03680099999,D5621,03781099999,03672099999,0377..."
3,"London, England, United Kingdom",2022-07-18,36.6,18.0,27.7,34.6,18.0,26.8,9.9,37.0,...,16.1,8,NaN,2022-07-18T05:04:33,2022-07-18T21:08:16,0.66,Clear,Clear conditions throughout the day.,clear-day,"03680099999,D5621,03781099999,03672099999,0377..."
4,"London, England, United Kingdom",2022-07-19,39.8,21.7,30.3,37.9,21.7,29.4,11.6,36.0,...,16.4,8,NaN,2022-07-19T05:05:49,2022-07-19T21:07:07,0.69,Rain,Clear conditions throughout the day with late ...,rain,"03680099999,D5621,03781099999,03672099999,D412..."
5,"London, England, United Kingdom",2022-07-20,26.6,19.1,22.7,26.6,19.1,22.7,15.4,64.9,...,14.3,6,NaN,2022-07-20T05:07:06,2022-07-20T21:05:55,0.75,"Rain, Partially cloudy",Partly cloudy throughout the day with rain.,rain,"03769099999,03680099999,D5621,03781099999,0367..."


## 2. Store Data
### Add File to GitHub

*Note: This is a quick & easy solution to host the data, but can also add file to IPFS, Arweave, GCP, AWS, etc...*

In [22]:
# Create File from Query results
results_df.to_csv("query_results.csv", index=False)

In [23]:
# Store file in GitHub repo
def git_add_commit_push(commit_message):
    try:
        subprocess.run(["git", "add", "."])
        subprocess.run(["git", "commit", "-m", commit_message])
        subprocess.run(["git", "push"])
        print("Git add, commit, and push successful!")
    except Exception as e:
        print("An error occurred:", e)


# Replace 'Your commit message here' with your desired commit message
commit_message = "Your commit message here"

git_add_commit_push(commit_message)

[main 1d372f1] Your commit message here
 2 files changed, 300 insertions(+), 34 deletions(-)
Git add, commit, and push successful!


To https://github.com/Demoniarc/ocean_data_nfts
   f116129..1d372f1  main -> main


## 3. Create Data NFT
### Add Data NFT to the OCEAN Protocol Marketplace

* Marketplace: https://market.oceanprotocol.com/
* SDK Docs: https://docs.oceanprotocol.com/developers/ocean.py/install

In [24]:
# Connect to Blockchain
connect_to_network("polygon-main")
config = get_config_dict("polygon-main")
ocean = Ocean(config)

In [25]:
# Connect to Wallet
accounts.clear()
private_key = os.getenv("PRIVATE_KEY")
wallet = accounts.add(private_key)
print("MATIC Balance:", wallet.balance() / 1e18)

MATIC Balance: 7.226968411279805


In [26]:
# Create Data NFT
name = "Weather Data NFT "

# Link to File
result = subprocess.run(
    ["git", "config", "--get", "user.name"], stdout=subprocess.PIPE, text=True
)
github_username = result.stdout.strip().replace(" ", "")
url = f"https://raw.githubusercontent.com/{github_username}/ocean_data_nfts/main/query_results.csv"

# Set Gas Fees
priority_fee = chain.priority_fee
max_fee = (chain.base_fee * 2) + priority_fee
gas_limit = int((priority_fee + max_fee) / 1e5)
print("priority_fee:", priority_fee, "max_fee:", max_fee, "gas_limit:", gas_limit)

# Configure Transaction Variables
tx_dict = {
    "from": wallet,
    "priority_fee": priority_fee,
    "max_fee": max_fee,
    "gas_limit": gas_limit,
}

# Publish Data NFT
(data_nft, datatoken, ddo) = ocean.assets.create_url_asset(name, url, tx_dict)

time.sleep(30)

# Set Price to Free
datatoken.create_dispenser(tx_dict)

print("Data NFT published:")
print(f"  data_nft: symbol={data_nft.symbol()}, address={data_nft.address}")
print(f"  datatoken: symbol={datatoken.symbol()}, address={datatoken.address}")
print(f"  did={ddo.did}")
print(f"  Link: https://market.oceanprotocol.com/asset/{ddo.did}")

priority_fee: 32093266514 max_fee: 191837331384 gas_limit: 2239305
Transaction sent: 0x669b8e12c23a978dc5920c8e2a439a56b1aba3b2578494d75d059aec952097f8
  Max fee: 191.837331384 gwei   Priority fee: 32.093266514 gwei   Gas limit: 2239305   Nonce: 7
  ERC721Factory.createNftWithErc20 confirmed   Block: 47140736   Gas used: 1595778 (71.26%)   Gas price: 120.108547443 gwei



INFO:ocean_lib.data_provider.data_encryptor:Asset urls encrypted successfully, encrypted urls str: 0x04a07c56b76caeb91f2270425718740804304ed89f8ec9028e41415667f8b21cb275a319db96b782f18f96aac7addddee699772b31d94166478dc933fa0ff27f7f88c2931f30e602119ceb40d81931c5d1bf394c0db4d3ea6ee13e486cae2c56512bb7211fb1eaa9c52443c6fe21fabf54b1f5c1ed624869f9b7b2b518cf06c5f6479983b37d9cb8cca76d233117b0cf9e137c33ce9c78df64dfe663f75195483fa6a72d9fe0c49b6c0c84829a5252ad49dcd57258b27d273a9d855fbe4cbc30c3331ce19f9679e4794be1e974cb4831377f8ae9918362cd9bcc40724b804f65a5b58725e564c543ebfd5f225e0cde00dba51f5883037f004fbca2f916ee8be0c162e7e1a8a98550893027db913a0511de530a7350c923728666a5eded04f7c13bc29f18729e89be68aa1aed2b349b926af38144e20c9e04becbc866a8007564f14f1c83aea0a22cb7d5, encryptedEndpoint https://v4.provider.polygon.oceanprotocol.com/api/services/encrypt?chainId=137
INFO:ocean_lib.data_provider.data_encryptor:Asset urls encrypted successfully, encrypted urls str: 0x04e26fb27ecd9d328c0339b5879e8918447455a

Transaction sent: 0xf4531a2f39366031bac5a7464a2aea659b2dec7134c427da0c24c58fd5498569
  Max fee: 191.837331384 gwei   Priority fee: 32.093266514 gwei   Gas limit: 2239305   Nonce: 8
  ERC721Template.setMetaData confirmed   Block: 47140740   Gas used: 212411 (9.49%)   Gas price: 127.440641717 gwei

Transaction sent: 0x4ea8649d679c026263f998fc56dc07d0254913177d48c1ff079cad57f74b8e6b
  Max fee: 191.837331384 gwei   Priority fee: 32.093266514 gwei   Gas limit: 2239305   Nonce: 9
  ERC20Template.createDispenser confirmed   Block: 47140763   Gas used: 283122 (12.64%)   Gas price: 131.037935405 gwei

Data NFT published:
  data_nft: symbol=Weather Data NFT , address=0xdbE4589621ed7b3178eD56bB38Ab07DCB08910d5
  datatoken: symbol=DT1, address=0xa564376aE5C2dd066BBA66F4D93998DCDf47cDFC
  did=did:op:ee7288202948bf35c19d084e58f5b87a7052d840d531b3d75ba0a6b0006d6b77
  Link: https://market.oceanprotocol.com/asset/did:op:ee7288202948bf35c19d084e58f5b87a7052d840d531b3d75ba0a6b0006d6b77
